https://www.bilibili.com/video/BV1AK4y1P7vs?p=1

# P1 层和块

## 两层的多层感知机

首先，回顾一下多层感知机

In [18]:
import torch
from torch import nn
from torch.nn import functional as F


# 这是一个两层的感知机，
net =nn.Sequential(nn.Linear(20,256),nn.ReLU(),nn.Linear(256,10))

# 随机生成输入数据  2是批量大小，现在数据一共就够1个批次，20是数据维度
X=torch.rand(2,20)

net(X)
# 输出就是2*10，因为是两个样本，每个样本输出是10维

tensor([[ 0.1633,  0.0985, -0.1644,  0.0371, -0.0613, -0.1588,  0.0566, -0.2984,
          0.1409,  0.1996],
        [ 0.0700,  0.1318, -0.1793, -0.0130, -0.1214, -0.0052,  0.1234, -0.1821,
          0.0853,  0.2966]], grad_fn=<AddmmBackward>)

+ 这里值得注意的是，`nn.Sequential`定义了一种特殊的`Module`。
+ 任何一个神经网络的层都应该是`Module`的子类
+ 不管是一层，还是多层的模型，都应该是`Module`的子类

## 继承Module模块来实现MLP

可以通过继承`Module`来自定义一些层/块
+ 最重要的需要实现的是两个函数，一个就是`__init__`函数，里面放着要定义的层的情况
+ 另一个就是forward函数，这个层的计算过程

In [5]:
class MLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.hidden=nn.Linear(20,256)
        self.out=nn.Linear(256,10)
        
    def forward(self,X):
        return self.out(F.relu(self.hidden(X)))

```python
dir(F)
help(F)
```
可以用上面的帮助看看，torch.nn.Functional里有很多激活函数

实例化多层感知机的层，然后在每次调用正向传播函数时调用这些层

In [8]:
net=MLP()
net(X)

tensor([[-0.0030, -0.1093, -0.0364, -0.1911, -0.0560, -0.1290, -0.0835, -0.1231,
         -0.0680,  0.1367],
        [-0.2054, -0.1313, -0.0451, -0.1426, -0.0807,  0.0965, -0.1160, -0.1647,
         -0.1683,  0.2031]], grad_fn=<AddmmBackward>)

## 顺序块（实现nn.Sequential）

可以用一个函数实现nn.Sequential的功能
+ 其中`*args`表示一个list of arguments，在上面的`nn.Sequential`调用时，传入的参数是一个网络层的列表，所以下面的*args其实也是一个网络层的列表

In [23]:
class MySequential(nn.Module):
    def __init__(self,*args):
        super().__init__()
        for block in args:
            self._modules[block]=block
            # 这是一个ordered dict 是有序的
    
    def forward(self,X):
        for block in self._modules.values():
            X=block(X)
        return X
    
    
net=MySequential(nn.Linear(20,256),nn.ReLU(),nn.Linear(256,10))
net(X)

tensor([[ 0.0631, -0.0432,  0.0557, -0.1294, -0.0249,  0.2181,  0.1399, -0.1292,
          0.1484,  0.1344],
        [ 0.1204, -0.0275,  0.1475, -0.1368, -0.0464,  0.1692,  0.1496, -0.2167,
          0.0604,  0.0680]], grad_fn=<AddmmBackward>)

可以看一下关于nn.Module.modules的一些说明，其实作用和上面的那个`self._modules`感觉差不多

In [21]:
help(nn.Module.modules)

Help on function modules in module torch.nn.modules.module:

modules(self) -> Iterator[ForwardRef('Module')]
    Returns an iterator over all modules in the network.
    
    Yields:
        Module: a module in the network
    
    Note:
        Duplicate modules are returned only once. In the following
        example, ``l`` will be returned only once.
    
    Example::
    
        >>> l = nn.Linear(2, 2)
        >>> net = nn.Sequential(l, l)
        >>> for idx, m in enumerate(net.modules()):
                print(idx, '->', m)
    
        0 -> Sequential(
          (0): Linear(in_features=2, out_features=2, bias=True)
          (1): Linear(in_features=2, out_features=2, bias=True)
        )
        1 -> Linear(in_features=2, out_features=2, bias=True)



## 在正向传播函数中执行代码

+ 如果Sequential不能满足你的需求的话，可以继承`nn.Module`来构建自己的网络
+ 主要的好处是在`__init__`和`forward`函数中，可以做大量自定义计算
+ 下面这个例子没有任何意义，只是为了表达，使用继承这种方式自定义的网络，可以进行非常高程度的定制。比如可以让某层的权重不参与训练，类似固定权重，预训练模型，前面的固定，只修改后几层的参数

In [24]:
class FixedHiddenMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.rand_weight=torch.rand((20,20),requires_grad=False)
        # 这里注意，这个权重是不参与训练的
        self.linear=nn.Linear(20,20)     
    
    def forward(self,X):
        X=self.linear(X)
        X=F.relu(torch.mm(X,self.rand_weight)+1)
        X=self.linear(X)
        while X.abs().sum()>1:
            X/=2
        return X.sum()

net=FixedHiddenMLP()
net(X)

tensor(-0.0077, grad_fn=<SumBackward0>)

+ 使用继承的这种方式，可以比使用Sequential方式更好满足自己的需求
+ 反向计算是不需要定义的，只要定义好前向，反向就可以根据前向自动求导

## 混合搭配各种组合块的方法

+ 不管是自己继承`nn.Module`得到的类，还是使用nn.Sequential得到的网络，其实都是nn.Module的子类，所以可以混合着用。
+ 底层是同一套逻辑
+ 网络层可以随便组，随便嵌套，疯狂套娃

In [25]:
class NestMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.net=nn.Sequential(nn.Linear(20,64),nn.ReLU(),
                              nn.Linear(64,32),nn.ReLU())
        self.linear=nn.Linear(32,16)
    
    def forward(self,X):
        return self.linear(self.net(X))

chimera=nn.Sequential(NestMLP(),nn.Linear(16,20),FixedHiddenMLP())
chimera(X)

tensor(-0.3004, grad_fn=<SumBackward0>)

-------------
https://www.bilibili.com/video/BV1AK4y1P7vs?p=2

# P2 参数管理

+ 假设已经定义好类了/模型结构了，该如何去访问参数
+ 还是以 具有单隐藏层的 多层感知机 为例

In [26]:
import torch
from torch import nn

net=nn.Sequential(nn.Linear(4,8),nn.ReLU(),nn.Linear(8,1))
X=torch.rand(size=(2,4))
net(X)

tensor([[-0.2548],
        [-0.2890]], grad_fn=<AddmmBackward>)

## 参数访问

## 把每层的权重都拿出来

In [27]:
net[2].state_dict()

OrderedDict([('weight',
              tensor([[ 0.0675,  0.2736,  0.0428,  0.1020, -0.1843,  0.0543, -0.1426, -0.1512]])),
             ('bias', tensor([-0.1674]))])

+ 这里net是使用`nn.Sequential`定义的，sequential其实就是类似list的一个东西，可以认为sequential就是python的list
+ 从自动机的角度来讲，每个层的权重就是每个层的状态，是会变的，所以这里访问参数和偏置的时候，调用的是`state_dict`

In [28]:
net

Sequential(
  (0): Linear(in_features=4, out_features=8, bias=True)
  (1): ReLU()
  (2): Linear(in_features=8, out_features=1, bias=True)
)

当然也可以直接访问特定的参数

In [31]:
net[2].weight

Parameter containing:
tensor([[ 0.0675,  0.2736,  0.0428,  0.1020, -0.1843,  0.0543, -0.1426, -0.1512]],
       requires_grad=True)

In [38]:
print(type(net[2].bias))
print(net[2].bias)
print(net[2].bias.data)
print(net[2].bias.grad)

<class 'torch.nn.parameter.Parameter'>
Parameter containing:
tensor([-0.1674], requires_grad=True)
tensor([-0.1674])
None


可以直接调用下面的命令，发现其实有bias和weight属性
```python
dir(net[2])
```

In [40]:
net[2].weight.grad==None

# 由于这里还没有进行反向计算，所以grad为None

True

## 也可以一次性访问所有参数

In [41]:
print(*[(name,param.shape) for name,param in net[0].named_parameters()])
print(*[(name,param.shape) for name,param in net.named_parameters()])

('weight', torch.Size([8, 4])) ('bias', torch.Size([8]))
('0.weight', torch.Size([8, 4])) ('0.bias', torch.Size([8])) ('2.weight', torch.Size([1, 8])) ('2.bias', torch.Size([1]))


+ 这里就会给出参数的名称，和对应参数的形状。
+ 如果在定义这些参数的时候，没有显示指定名称，则就默认 `#num.weight/bias`，层的名称加上是权重还是偏置，就可以了
+ ReLU，也就是序号为1的层，没有权重，所以没有参数，取不出来东西
+ 另外，有了名字就可以用名字来获取参数的信息

In [42]:
net.state_dict()["2.bias"].data

tensor([-0.1674])

In [43]:
print([(name,param.shape) for name,param in net[0].named_parameters()])
print([(name,param.shape) for name,param in net.named_parameters()])

[('weight', torch.Size([8, 4])), ('bias', torch.Size([8]))]
[('0.weight', torch.Size([8, 4])), ('0.bias', torch.Size([8])), ('2.weight', torch.Size([1, 8])), ('2.bias', torch.Size([1]))]


可以看到，这部分代码没有前面的`*`号也可以运行，只是输出结果会是列表，而不是上面的元组。

## 从嵌套块收集参数

In [49]:
def block1():
    return nn.Sequential(nn.Linear(4,8),nn.ReLU(),nn.Linear(8,4),nn.ReLU())

def block2():
    net =nn.Sequential()
    for i in range(4):
        net.add_module(f'block {i}',block1())
        # 第一个参数是name，第二个参数是网络层module
    return net

rgnet=nn.Sequential(block2(),nn.Linear(4,1))
rgnet(X)

tensor([[0.6426],
        [0.6423]], grad_fn=<AddmmBackward>)

In [50]:
rgnet

Sequential(
  (0): Sequential(
    (block 0): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 1): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 2): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 3): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
  )
  (1): Linear(in_features=4, out_features=1, bias=True)
)

+ 可以看到，嵌套时候的网络表示，稍微复杂了一点，也嵌套了
+ 这个时候如果想从嵌套块收集参数，就要像json嵌套访问一样，一层一层去看
+ 最外层是 0→block2,1→linear
+ 0之后就是block2
+ 注意，这里只支持数字访问，不支持module的name的str访问

In [60]:
print(rgnet[0][0])
print(rgnet[0][0][0])

Sequential(
  (0): Linear(in_features=4, out_features=8, bias=True)
  (1): ReLU()
  (2): Linear(in_features=8, out_features=4, bias=True)
  (3): ReLU()
)
Linear(in_features=4, out_features=8, bias=True)


+ 模型比较复杂的时候，看起来就会比较费劲
+ 所以需要设计好模块化的东西，这样对自己会比较方便

In [59]:
net.state_dict()["2.bias"].data

tensor([-0.1674])

## 内置初始化

In [9]:
def init_normal(m):
    if type(m)==nn.Linear:
        nn.init.normal_(m.weight,mean=0,std=0.01) #Fills the input Tensor with values drawn from the normal
distribution
        nn.init.zeros_(m.bias) #Fills the input Tensor with the scalar value `0`.

        
net=nn.Sequential(nn.Linear(4,8),nn.ReLU(),nn.Linear(8,1))
X=torch.rand(size=(2,4))
net(X)

print(f"初始化前的参数，权重数值：{net[0].weight.data[0]},偏置数值：{net[0].bias.data[0]}")
net.apply(init_normal)
print(f"初始化后的参数，权重数值：{net[0].weight.data[0]},偏置数值：{net[0].bias.data[0]}")

初始化前的参数，权重数值：tensor([-0.1643,  0.3355, -0.4695, -0.0503]),偏置数值：-0.3344457745552063
初始化后的参数，权重数值：tensor([-0.0086, -0.0010,  0.0026,  0.0192]),偏置数值：0.0


pytorch中函数名字中的下划线写在后面，表示这是一个原地操作的函数，不会返回东西，而是直接对变量进行修改

初始化的时候，除了可以初始化成一个分布，也可以初始化成一个常量，反正就是也可以自己定制 
+ 单纯的API可以这样操作，但是从算法角度来说不可以这样做，因为如果w都是1，其实就相当于没有。。。

In [10]:
def init_constant(m):
    if type(m)==nn.Linear:
        nn.init.constant_(m.weight,1)
        nn.init.zeros_(m.bias)
        
print(f"初始化前的参数，权重数值：{net[0].weight.data[0]},偏置数值：{net[0].bias.data[0]}")
net.apply(init_constant)
print(f"初始化后的参数，权重数值：{net[0].weight.data[0]},偏置数值：{net[0].bias.data[0]}")

初始化前的参数，权重数值：tensor([-0.0086, -0.0010,  0.0026,  0.0192]),偏置数值：0.0
初始化后的参数，权重数值：tensor([1., 1., 1., 1.]),偏置数值：0.0


## 对某些块应用不同的初始化方法

In [11]:
def xavier(m):
    if type(m)==nn.Linear:
        nn.init.xavier_uniform_(m.weight)
def init_42(m):
    if type(m)==nn.Linear:
        nn.init.constant_(m.weight,42)

net[0].apply(xavier)
net[2].apply(init_42)

print(net[0].weight.data[0])
print(net[2].weight.data)

tensor([-0.5871,  0.4554, -0.4848,  0.0134])
tensor([[42., 42., 42., 42., 42., 42., 42., 42.]])


In [12]:
net[0].weight.shape # 刚好和上面定义的 输入4，输入8是转置的，矩阵乘法

torch.Size([8, 4])

## 自定义初始化

可以想怎么定义，就怎么定义，哪怕是一些奇奇怪怪的都可以，当然只是说可以这样做，但是这样做不一定好，所以没必要这么做

In [14]:
def my_init(m):
    if type(m)==nn.Linear:
        print("Init",*[(name,param.shape) for name,param in m.named_parameters()][0])
        nn.init.uniform_(m.weight,-10,10)
        m.weight.data*=m.weight.data.abs() >=5
        print(m.weight.data.abs())
        print(m.weight.data)
        

net.apply(my_init)
net[0].weight[:2]

Init weight torch.Size([8, 4])
tensor([[5.3637, 0.0000, 5.4437, 0.0000],
        [0.0000, 0.0000, 0.0000, 6.8892],
        [0.0000, 0.0000, 0.0000, 8.2707],
        [0.0000, 0.0000, 0.0000, 6.2146],
        [0.0000, 9.8965, 6.5613, 9.2368],
        [7.6442, 5.9515, 5.7334, 0.0000],
        [0.0000, 6.9959, 6.1654, 9.0690],
        [5.2384, 0.0000, 0.0000, 8.4945]])
tensor([[-5.3637,  0.0000, -5.4437, -0.0000],
        [-0.0000,  0.0000,  0.0000,  6.8892],
        [-0.0000, -0.0000, -0.0000, -8.2707],
        [-0.0000, -0.0000,  0.0000,  6.2146],
        [-0.0000, -9.8965, -6.5613,  9.2368],
        [ 7.6442,  5.9515,  5.7334,  0.0000],
        [ 0.0000, -6.9959, -6.1654,  9.0690],
        [ 5.2384, -0.0000, -0.0000, -8.4945]])
Init weight torch.Size([1, 8])
tensor([[0.0000, 0.0000, 0.0000, 8.1006, 7.1980, 7.2760, 0.0000, 6.8748]])
tensor([[-0.0000, -0.0000, -0.0000,  8.1006,  7.1980, -7.2760,  0.0000, -6.8748]])


tensor([[-5.3637,  0.0000, -5.4437, -0.0000],
        [-0.0000,  0.0000,  0.0000,  6.8892]], grad_fn=<SliceBackward>)

In [31]:
import numpy as np
a=np.array([1, -2, 3])
a*=np.abs(a) >=2
a

array([ 0, -2,  3])

In [32]:
a=np.array([1, -2, 2])
a*=np.abs(a) >=0
a

array([ 1, -2,  2])

对于上面的运算，先比较大小，再自乘。
`a>=0`，三个值的绝对值都满足，所以否返回1，a*[1,1,1]还是等于本身

In [29]:
a=1
a*=2
a

2

In [30]:
a=1
a*=2>3
a

0

+ 上面这个运算会产生疑惑主要是因为搞不清楚 比较运算和自乘运算 这二者的优先级
+ 代码验证之后，可以发现，先进行比较运算，得到0或者1，然后再自乘

甚至可以直接把权重拿出来加加减减

In [33]:
net[0].weight.data[:]+=1
net[0].weight.data[0,0]=42
net[0].weight.data[0]

tensor([42.0000,  1.0000, -4.4437,  1.0000])

其实就是个数，不要恐惧，可以进行数的所有操作，只是没必要这样

## 参数绑定

算是一种对参数的小应用吧，比较常见的一种小技巧？？

+ 如果我们想在一些层之间share parameter共享参数
+ 比如有两个数据流进来，想要共享权重，那么可以考虑使用参数绑定
    + 其实就是让那两个参数指向同一片内存区域。。。
+ 实现的时候，其实就是把要共享的那个层先构造出来，放到合适的位置就可以了

In [35]:
shared = nn.Linear(8,8)
net=nn.Sequential(nn.Linear(4,8),nn.ReLU(),shared,nn.ReLU(),shared,
                 nn.ReLU(),nn.Linear(8,1))
net(X)

print(net[2].weight.data[0]==net[4].weight.data[0])
net[2].weight.data[0,0]=100
print(net[2].weight.data[0]==net[4].weight.data[0])

tensor([True, True, True, True, True, True, True, True])
tensor([True, True, True, True, True, True, True, True])


这个共享权重的方式看起来好像挺合理的，其实就相当于一波里这层调了两次，反正只要一直去调，迟早能调出让两个地方都满意的参数来

----
https://www.bilibili.com/video/BV1AK4y1P7vs?p=3

# P3 自定义层

+ 之前说了自定义网络，使用`nn.Sequential`，这里来说一下怎么自定义层。
+ 其实自定义层和自定义网络看起来差不多，因为层也是`nn.module`的一个子类

比如：构建一个没有任何参数的自定义层

In [1]:
import torch
import torch.nn.functional as F
from torch import nn

class CenteredLayer(nn.Module):
    def __init__(self):
        super().__init__()
    
    def forward(self,X):
        return X-X.mean()

layer=CenteredLayer()
layer(torch.FloatTensor([1,2,3,4,5]))

tensor([-2., -1.,  0.,  1.,  2.])

然后将层作为组件合并到构建更复杂的模型中

In [2]:
net=nn.Sequential(nn.Linear(8,128),CenteredLayer())

Y=net(torch.rand(4,8))
Y.mean()

tensor(9.3132e-09, grad_fn=<MeanBackward0>)

这里很明显，如果我构建一个以`CenteredLayer`为输出层的一个网络，rand生成的是[0,1)之间的数字，均值就是0.5，对这些数字，X-X.mean，然后再对Y进行mean，最后基本应该就是0了，会有一些浮点数的误差

---
当然，也可以定义带参数的图层

In [4]:
class MyLinear(nn.Module):
    def __init__(self,in_uints,units):
        super().__init__()
        self.weight=nn.Parameter(torch.randn(in_uints,units))
        self.bias=nn.Parameter(torch.randn(units,))
        # 这里注意，这个units 后面有逗号创建的是列向量，没有就是行向量
    def forward(self,X):
        linear=torch.matmul(X,self.weight.data)+self.bias.data
        return F.relu(linear)
    
dense=MyLinear(5,3)
dense.weight

Parameter containing:
tensor([[-0.4501,  0.2334, -0.0847],
        [-0.6307, -0.6267, -0.5578],
        [-0.8559, -0.5199,  0.8293],
        [-0.5777,  1.6209,  0.0183],
        [-0.5251,  0.6346, -0.7452]], requires_grad=True)


2:19

参数都是Parameter这个类的实例，所以如果要定义带有参数的图层，就需要定义Parameter类的参数

**使用自定义层直接执行正向传播计算**

In [5]:
dense(torch.rand(2,5))

tensor([[0.0000, 2.3223, 0.5695],
        [0.0000, 2.8477, 0.9815]])

这里解释一下，dense层是输入5，输出3，所以2个样本，输出就是2*3

**使用自定义层构建模型**

In [6]:
net=nn.Sequential(MyLinear(64,8),MyLinear(8,1))
net(torch.rand(2,64))

tensor([[7.0712],
        [0.0000]])

-----
https://www.bilibili.com/video/BV1AK4y1P7vs?p=4

# P4 读写文件

加载和保存张量

In [7]:
import torch
from torch import nn
from torch.nn import functional as F

x=torch.arange(4)
torch.save(x,'x-file')  # 这个不需要后缀名

x2=torch.load("x-file")
x2

tensor([0, 1, 2, 3])

存储一个张量列表，然后把它们读回内存

In [9]:
y=torch.zeros(4)
torch.save([x,y],'x-files')

x2,y2=torch.load('x-files')
(x2,y2)

(tensor([0, 1, 2, 3]), tensor([0., 0., 0., 0.]))

In [10]:
a=torch.load('x-files')
a

[tensor([0, 1, 2, 3]), tensor([0., 0., 0., 0.])]

写入或读取从字符串映射到张量的字典

In [11]:
mydict={'x':x,'y':y}
torch.save(mydict,'mydict')
mydict2=torch.load('mydict')
mydict2

{'x': tensor([0, 1, 2, 3]), 'y': tensor([0., 0., 0., 0.])}

看下来。。。感觉挺牛逼的，啥都可以存啊

**加载和保存模型参数**

In [14]:
class MLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.hidden=nn.Linear(20,256)
        self.output=nn.Linear(256,10)
    def forward(self,x):
        return self.output(F.relu(self.hidden(x)))
    
net=MLP()
X=torch.rand(size=(2,20))
Y=net(X)

将模型的参数存储为一个叫做`mpl.params`的文件

In [21]:
net.state_dict()['output.bias']
# state_dict是一个词典，上面torch也是可以直接save词典的
# 而神经网络模型的权重在pytorch中是以词典的形式存在的

tensor([ 0.0519, -0.0371,  0.0142,  0.0538,  0.0209,  0.0098, -0.0225, -0.0120,
         0.0362, -0.0483])

In [22]:
torch.save(net.state_dict(),'mlp.params')

+ 我们都知道神经网络有网络层的结构（拓扑结构），还有参数（数值），还有梯度（训练完可能就消失了，不过计算过程中，全程驻在内存的是叶子结点的梯度）等
+ 而pytorch中，使用的是 Imperative style，
+ 不同于tensorflow先构建图，编译静态图之后再去执行。
+ pytorch是命令式的，非常pythonic的风格，解释性语言，每句都可以执行，虽然方便调试，但是其实并不方便把整个模型存下来
+ 所以和tensorflow和mxnet不一样，很难把整个模型的定义给存下来（因为它这个是可以分步执行的，所以很难判断模型结构的起点和终点）
+ 但是也有别的方法可以存，比如torchscript可以存
    + 参考：[pytorch部署torchscript篇](https://www.cnblogs.com/allentbky/p/14163898.html)
    + [【Pytorch部署】TorchScript](https://zhuanlan.zhihu.com/p/135911580)
    + [TorchScript简介](https://www.jianshu.com/p/a94d49351e05)
+ 所以**在pytorch中，默认只能存权重，计算（模型定义）存不下来**

关于pytorch框架的说明，其实有个论文
+ [PyTorch: An Imperative Style, High-Performance Deep Learning Library](https://arxiv.org/abs/1912.01703)
+ 论文解读：[PyTorch: An Imperative Style, High-Performance Deep Learning Library](https://www.cnblogs.com/RyanXing/p/PyTorch_paper.html)

读论文的两个工具推荐：
+ https://www.ivysci.com/
+ https://readpaper.com/

**加载和使用一个已经保存过的权重**

In [24]:
clone = MLP()
# 重新构建模型结构，模型的定义要有

clone.load_state_dict(torch.load('mlp.params'))
# 加载刚刚保存的参数（因为要做参数名的匹配，所以不再使用load函数，而是使用load_state_dict）

clone.eval()

MLP(
  (hidden): Linear(in_features=20, out_features=256, bias=True)
  (output): Linear(in_features=256, out_features=10, bias=True)
)

验证一下，这个存的权重恢复出来的网络是不是和刚刚一样

In [26]:
Y_clone=clone(X)
# 输入同样的值，看输出是否形态
Y_clone==Y

tensor([[True, True, True, True, True, True, True, True, True, True],
        [True, True, True, True, True, True, True, True, True, True]])

-----
# P5 Q&A